In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
cmap = mpl.colormaps.get_cmap('tab10').colors
plt.style.use('../l3.mplstyle')
import pydicom 
import os

# SNR

In [ ]:
with pydicom.dcmread('C:/Users/steph/Downloads/MRT/T1WEIGHTED_IMAGE_TSE_0002/902.MR.PRAKTIKUM_SCAN_PROTOCOL.0002.0011.2024.06.26.11.29.54.49682.156687664.IMA') as image:
    arr = image.pixel_array
    # Use same vmin, vmax for low and high SNR image for better comparison
    vmin, vmax = image.SmallestImagePixelValue, image.LargestImagePixelValue 
# Determine SNR:
x_on, y_on = 153, 217
x_off, y_off = 10,10
width = 80
roi_on = arr[y_on:y_on+width, x_on:x_on+width]
roi_off = arr[y_off:y_off+width, x_off:x_off+width]
snr = np.mean(roi_on)/np.mean(roi_off)
print(f'SNR: {snr}')
# Plotting
fig,ax = plt.subplots()
im = ax.imshow(arr, cmap='gray', vmin=vmin, vmax=vmax)
rect_on = plt.Rectangle((x_on,y_on),width,width, edgecolor=cmap[0], facecolor='none')
rect_off = plt.Rectangle((x_off,y_off),width,width, edgecolor=cmap[1], facecolor='none')
ax.add_patch(rect_on)
ax.add_patch(rect_off)
ax.set(xticks=[], yticks=[])
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('Intensität [ADU]')
fig.tight_layout()
fig.savefig('plots/t1_high_snr.pdf', bbox_inches='tight')

# ------------------------------------- Low SNR -------------------------------------

with pydicom.dcmread('C:/Users/steph/Downloads/MRT/T1_WEIGHTED_LOWRES_0003/902.MR.PRAKTIKUM_SCAN_PROTOCOL.0003.0011.2024.06.26.11.29.54.49682.156689714.IMA') as image: 
    arr = image.pixel_array
# Determine SNR:
x_on, y_on = 210, 310
x_off, y_off = 10,10
width = 100
roi_on = arr[y_on:y_on+width, x_on:x_on+width]
roi_off = arr[y_off:y_off+width, x_off:x_off+width]
snr = np.mean(roi_on)/np.mean(roi_off)
print(f'SNR: {snr}')
# Plotting
fig,ax = plt.subplots()
im = ax.imshow(arr, cmap='gray', vmin=vmin, vmax=vmax) 
rect_on = plt.Rectangle((x_on,y_on),width,width, edgecolor=cmap[0], facecolor='none')
rect_off = plt.Rectangle((x_off,y_off),width,width, edgecolor=cmap[1], facecolor='none')
ax.add_patch(rect_on)
ax.add_patch(rect_off)
ax.set(xticks=[], yticks=[])
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('Intensität [ADU]')
fig.tight_layout()
fig.savefig('plots/t1_low_snr.pdf', bbox_inches='tight')

# T1 vs T2

In [ ]:
# Choose one coronal slice for plotting
with pydicom.dcmread('C:/Users/steph/Downloads/MRT/T1WEIGHTED_IMAGE_TSE_0002/902.MR.PRAKTIKUM_SCAN_PROTOCOL.0002.0011.2024.06.26.11.29.54.49682.156687664.IMA') as image_t1:
    arr_t1 = image_t1.pixel_array

# Determine Michelson-contrast of it:
x_bright, y_bright = 150, 220
x_dark, y_dark = 210, 330
width = 80
roi_bright = arr_t1[y_bright:y_bright+width, x_bright:x_bright+width]
roi_dark = arr_t1[y_dark:y_dark+width, x_dark:x_dark+width]
contrast = (np.mean(roi_bright)-np.mean(roi_dark))/(np.mean(roi_bright)+np.mean(roi_dark))
print(f'Contrast t1: {contrast}')
# Get transversal and sagittal slices
basepath = 'C:/Users/steph/Downloads/MRT/T1WEIGHTED_IMAGE_TSE_0002'
paths = [os.path.join(basepath, relpath) for relpath in os.listdir(basepath)]
for i, path in enumerate(paths):
    with pydicom.dcmread(path) as image:
        arr = image.pixel_array
    row = arr[250,:]
    column = arr[:,250]
    if i == 0:
        # First iteration, therefore need to initialise h_slice, v_slice arrays with the right sizes
        h_slice = row
        v_slice = column
    else:
        h_slice = np.vstack((h_slice, row))
        v_slice = np.vstack((v_slice, column))
v_slice = v_slice.T
# Plotting:
grid = gridspec.GridSpec(2,2, height_ratios=[1,1])
fig = plt.figure(figsize=(8,8))

ax = fig.add_subplot(grid[0,0])
im_t1 = ax.imshow(arr_t1, cmap='gray')
ax.set(xticks=[], yticks=[])
# Plot rects for contrast
rect_bright = plt.Rectangle((x_bright,y_bright),width,width, edgecolor=cmap[0], facecolor='none')
rect_dark = plt.Rectangle((x_dark,y_dark),width,width, edgecolor=cmap[1], facecolor='none')
ax.add_patch(rect_bright)
ax.add_patch(rect_dark)
ax.set_title('Coronal')
# Plot lines
ax.axvline(250, color='red', linestyle='dashed')
ax.axhline(250, color='green', linestyle='--')

ax = fig.add_subplot(grid[0,1])
ax.imshow(v_slice, cmap='gray')
ax.set_aspect(np.shape(v_slice)[1]/np.shape(v_slice)[0]) # Make square
ax.set(xticks=[], yticks=[])
ax.patch.set_edgecolor('red')
ax.patch.set_linewidth(5)
ax.patch.set_linestyle('dashed')  
ax.set_title('Sagittal')

ax = fig.add_subplot(grid[1,0])
ax.imshow(h_slice, cmap='gray')
ax.set_aspect(np.shape(h_slice)[1]/np.shape(h_slice)[0]) # Make square
ax.set(xticks=[], yticks=[])
ax.patch.set_edgecolor('green')
ax.patch.set_linewidth(5)
ax.patch.set_linestyle('dashed') 
ax.set_title('Transversal')

ax = fig.add_axes([0.65, 0.05, 0.05, 0.4])
cbar = fig.colorbar(im_t1, cax=ax)
cbar.set_label('Intensität [ADU]')

fig.savefig('plots/t1.pdf')

### T2

In [ ]:
# Choose one coronal slice for plotting
with pydicom.dcmread('C:/Users/steph/Downloads/MRT/T2_WEIGTHED_0004\902.MR.PRAKTIKUM_SCAN_PROTOCOL.0004.0011.2024.06.26.11.29.54.49682.156691663.IMA') as image_t2:
    arr_t2 = image_t2.pixel_array

# Determine Michelson-contrast:
x_bright, y_bright = 45,86
x_dark, y_dark = 26,44
width = 28
roi_bright = arr_t2[y_bright:y_bright+width, x_bright:x_bright+width]
roi_dark = arr_t2[y_dark:y_dark+width, x_dark:x_dark+width]
contrast = (np.mean(roi_bright)-np.mean(roi_dark))/(np.mean(roi_bright)+np.mean(roi_dark))
print(f'Contrast t2: {contrast}')

# Get transversal and sagittal slices
basepath = 'C:/Users/steph/Downloads/MRT/T2_WEIGTHED_0004'
paths = [os.path.join(basepath, relpath) for relpath in os.listdir(basepath)]
for i, path in enumerate(paths):
    with pydicom.dcmread(path) as image:
        arr = image.pixel_array
    row = arr[64,:]
    column = arr[:,64]
    if i == 0:
        # First iteration, therefore need to initialise h_slice, v_slice arrays with the right sizes
        h_slice = row
        v_slice = column
    else:
        h_slice = np.vstack((h_slice, row))
        v_slice = np.vstack((v_slice, column))
v_slice = v_slice.T
# Plotting:
grid = gridspec.GridSpec(2,2, height_ratios=[1,1])
fig = plt.figure(figsize=(8,8))

ax = fig.add_subplot(grid[0,0])
im_t1 = ax.imshow(arr_t2, cmap='gray')
ax.set(xticks=[], yticks=[])
# Plot rects for contrast
rect_bright = plt.Rectangle((x_bright,y_bright),width,width, edgecolor=cmap[0], facecolor='none')
rect_dark = plt.Rectangle((x_dark,y_dark),width,width, edgecolor=cmap[1], facecolor='none')
ax.add_patch(rect_bright)
ax.add_patch(rect_dark)
ax.set_title('Coronal')
# Plot lines
ax.axvline(64, color='red', linestyle='dashed')
ax.axhline(64, color='green', linestyle='--')

ax = fig.add_subplot(grid[0,1])
ax.imshow(v_slice, cmap='gray')
ax.set_aspect(np.shape(v_slice)[1]/np.shape(v_slice)[0]) # Make square
ax.set(xticks=[], yticks=[])
ax.patch.set_edgecolor('red')
ax.patch.set_linewidth(5)
ax.patch.set_linestyle('dashed')  
ax.set_title('Sagittal')

ax = fig.add_subplot(grid[1,0])
ax.imshow(h_slice, cmap='gray')
ax.set_aspect(np.shape(h_slice)[1]/np.shape(h_slice)[0]) # Make square
ax.set(xticks=[], yticks=[])
ax.patch.set_edgecolor('green')
ax.patch.set_linewidth(5)
ax.patch.set_linestyle('dashed') 
ax.set_title('Transversal')

ax = fig.add_axes([0.65, 0.05, 0.05, 0.4])
cbar = fig.colorbar(im_t1, cax=ax)
cbar.set_label('Intensität [ADU]')

fig.savefig('plots/t2.pdf')

# Artefacts

In [ ]:
with pydicom.dcmread('C:/Users/steph/Downloads/MRT/T1WEIGHTED_IMAGE_ARTIFACT_0005\902.MR.PRAKTIKUM_SCAN_PROTOCOL.0005.0008.2024.06.26.11.29.54.49682.156693796.IMA') as image:
    arr = image.pixel_array
fig,ax = plt.subplots()
im = ax.imshow(arr, cmap='gray')
ax.set(xticks=[], yticks=[])
cbar = fig.colorbar(im, ax=ax)
cbar.set_label('Intensität [ADU]')
fig.tight_layout()
fig.savefig('plots/artifact.pdf', bbox_inches='tight')

# Vegetables

In [ ]:
#Choose one sagittal slice for plotting
with pydicom.dcmread('C:/Users/steph/Downloads/MRT/T1WEIGHTED_IMAGE_TSE_0017/902.MR.PRAKTIKUM_SCAN_PROTOCOL.0017.0012.2024.06.26.11.29.54.49682.156724687.IMA') as image:
    arrt1 = image.pixel_array



# Get coronal and transversal slices
basepath = 'C:/Users/steph/Downloads/MRT/T1WEIGHTED_IMAGE_TSE_0017'
paths = [os.path.join(basepath, relpath) for relpath in os.listdir(basepath)]
for i, path in enumerate(paths):
    with pydicom.dcmread(path) as image:
        arr = image.pixel_array
    row = arr[330,:]
    column = arr[:,280]
    if i == 0:
        # First iteration, therefore need to initialise h_slice, v_slice arrays with the right sizes
        h_slice = row
        v_slice = column
    else:
        h_slice = np.vstack((h_slice, row))
        v_slice = np.vstack((v_slice, column))
v_slice = v_slice.T
# Plotting:
grid = gridspec.GridSpec(2,2, height_ratios=[1,1])
fig = plt.figure(figsize=(8,8))

ax = fig.add_subplot(grid[0,0])
im_t1 = ax.imshow(arrt1, cmap='gray')
ax.set(xticks=[], yticks=[])
ax.set_title('Sagittal')
# Plot lines
ax.axvline(280, color='red', linestyle='dashed')
ax.axhline(330, color='green', linestyle='--')

ax = fig.add_subplot(grid[0,1])
ax.imshow(v_slice, cmap='gray')
ax.set_aspect(np.shape(v_slice)[1]/np.shape(v_slice)[0]) # Make square
ax.set(xticks=[], yticks=[])
ax.patch.set_edgecolor('red')
ax.patch.set_linewidth(4)
ax.patch.set_linestyle('dashed')  
ax.set_title('Transveral')

ax = fig.add_subplot(grid[1,0])
ax.imshow(h_slice, cmap='gray')
ax.set_aspect(np.shape(h_slice)[1]/np.shape(h_slice)[0]) # Make square
ax.set(xticks=[], yticks=[])
ax.patch.set_edgecolor('green')
ax.patch.set_linewidth(5)
ax.patch.set_linestyle('dashed') 
ax.set_title('Coronal')

ax = fig.add_axes([0.65, 0.05, 0.05, 0.4])
cbar = fig.colorbar(im_t1, cax=ax)
cbar.set_label('Intensität [ADU]')

fig.savefig('plots/veggie.pdf')